In [1]:
# !pip install -U pandas pinecone-client sentence-transformers


# Dense Retriever

This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.



The project aims to train sentence embedding models on very large sentence level datasets using a self-supervised contrastive learning objective. We used the pretrained nreimers/MiniLM-L6-H384-uncased model and fine-tuned in on a 1B sentence pairs dataset. We use a contrastive learning objective: given a sentence from the pair, the model should predict which out of a set of randomly sampled other sentences, was actually paired with it in our dataset.




[Doc](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

In [2]:
from sentence_transformers import SentenceTransformer

import datasets
import rich
from IPython.display import Image, JSON
from IPython.core.display import HTML
import numpy as np

from transformers import AutoTokenizer


In [3]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'

In [4]:

# Initialize retriever with SentenceTransformer model 
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Tokenizer

In [5]:
text = "What does 'super' do in Python? - difference between super().__init__() and explicit superclass __init__()"
#text = "meN shoes running ran"

resp = model.encode(text, output_value=None)

rich.print(resp)

{
    'input_ids': tensor([  101,  2054,  2515,  1005,  3565,  1005,  2079,  1999, 18750,  1029,
         1011,  4489,  2090,  3565,  1006,  1007,  1012,  1035,  1035,  1999,
         4183,  1035,  1035,  1006,  1007,  1998, 13216,  3565, 26266,  1035,
         1035,  1999,  4183,  1035,  1035,  1006,  1007,   102]),
    'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    'token_embeddings': tensor([[-0.4915,  0.0079,  0.1150,  ...,  0.3318, -0.1832, -0.0306],
        [-0.6559,  0.0482,  0.0829,  ...,  0.3092,  0.2388, -0.0171],
        [-0.7676, -0.0600,  0.0628,  ...,  0.4701,  0.5587, -0.2076],
        ...,
        [-0.5866,  0.3504,  0.1313,  ...,  0.5132,  0.6868,  0.3820],
        [-0.4603, -0.0785,  0.1447,  ...,  0.3814,  0.2534,  0.0358],
        [-0.6221, -0.1158,  0.1800,  ...,  0.5810,  0.3475, -0.0104]]),
    'sentence_embedding': tensor([-1.4246e-01,  3.0715e-03,  1.5940e-02,  2.8173e-02,  2.1864e-02,
        -8.6956e-02, -3.7082e-02,  8.4534e-02, -8.4545e-02, -1.9203e-02,
         1.2949e-02,  1.1646e-01, -6.2878e-03,  5.2016e-03, -4.4650e-02,
         5.4340e-04,  1.1038e-01, -8.6887e-02, -7.3036e-02, -1.9409e-02,
         1.4030e-01, -2.7500e-02, -6.8370e-02,  5.3648e-02, -1.0747e-02,
        -3.9285e-02, -7.5669e-02,  2.1057e-02,  3.5978e-02,  6.1648e-02,
        -3.6488e-02,  1.1507e-01, -4.9677e-03,  3.0514e-02, -7.8639e-02,
         9.8421e-02,  2.5322e-02, -1.0002e-02,  4.8957e-02, -7.0811e-02,
        -1.3952e-02, -1.9013e-02, -3.3804e-02, -1.4941e-02,  3.1447e-02,
         4.5754e-02, -3.8936e-02,  1.0241e-02, -4.7750e-02, -7.2783e-02,
        -1.9285e-02,  5.6163e-02, -6.3321e-03,  1.9977e-02,  2.6490e-02,
         6.2586e-02, -1.2018e-03, -8.9426e-02, -4.6173e-02, -3.6813e-02,
        -4.8190e-02,  2.1556e-02,  1.9962e-02,  3.5407e-02,  2.6642e-02,
        -5.8311e-02, -1.2200e-03, -3.4401e-03,  1.7953e-02,  5.4156e-02,
        -5.0989e-03, -2.7924e-02,  5.4463e-02,  1.1905e-01,  1.2868e-01,
        -8.8595e-02, -1.6164e-02, -3.2878e-02, -3.5485e-02, -1.9966e-02,
         3.9049e-02, -4.9114e-02,  4.5685e-02, -1.2991e-02,  4.6847e-02,
        -5.6101e-02,  6.8093e-02,  2.4826e-03,  2.6495e-02,  7.6566e-02,
         1.5663e-02, -6.9162e-02, -6.2864e-02,  2.0814e-02, -1.0910e-01,
         2.1203e-02, -1.4046e-02,  1.5385e-02, -6.3359e-03, -2.8475e-02,
         1.3662e-02, -1.3258e-01, -1.8202e-02, -2.1830e-02,  8.3101e-02,
        -6.8789e-02, -9.0988e-03, -7.0356e-02,  6.2706e-02, -4.4722e-02,
        -7.2774e-03, -1.7467e-01,  2.3604e-02, -6.0021e-02, -3.4137e-02,
         7.4124e-02,  4.0557e-03,  5.9823e-02,  1.4840e-03, -4.6625e-03,
         7.0497e-03,  7.7516e-02,  7.3966e-02,  3.4246e-02, -4.2010e-02,
         8.9824e-02, -1.0567e-01, -6.4156e-33, -1.6688e-02, -3.7029e-02,
         1.5546e-03, -1.9020e-02, -3.0639e-02, -3.2764e-02,  1.1822e-02,
         9.7167e-03,  1.2093e-04,  7.0550e-02,  2.4579e-02,  5.7831e-02,
         6.8307e-02,  5.6851e-02,  7.3616e-02,  2.2824e-02, -7.3005e-02,
        -1.5891e-02,  2.0759e-02, -1.2286e-02,  6.4947e-02,  1.2431e-01,
         1.8076e-02,  1.1093e-02, -2.1620e-04, -1.7142e-02,  1.4853e-02,
        -1.2691e-02, -5.0507e-02,  3.7222e-02, -4.0360e-02, -7.4048e-02,
        -1.0970e-02,  4.1702e-02, -6.8921e-02,  4.1219e-03,  1.4185e-02,
        -7.5804e-02, -1.1405e-02, -8.4661e-02, -3.4637e-02,  4.0803e-02,
        -1.9901e-02, -1.4428e-02, -3.2878e-02, -1.3772e-02, -1.9814e-02,
         9.5046e-02,  9.0082e-03, -2.6932e-02,  1.6957e-02, -2.3650e-02,
         3.9388e-02,  1.2614e-02, -1.8223e-02,  8.4496e-02,  1.0471e-01,
         7.0581e-02,  2.9548e-02, -6.3651e-02,  5.4294e-04, -6.2097e-02,
         1.1686e-02,  6.8007e-02, -4.1757e-02,  5.4762e-05,  7.2575e-03,
         5.3216e-02,  3.9076e-02,  3.3343

In [6]:
resp['token_embeddings'].shape # for each token, we have embedding


torch.Size([38, 384])

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [8]:
tokenizer(text)


{'input_ids': [101, 2054, 2515, 1005, 3565, 1005, 2079, 1999, 18750, 1029, 1011, 4489, 2090, 3565, 1006, 1007, 1012, 1035, 1035, 1999, 4183, 1035, 1035, 1006, 1007, 1998, 13216, 3565, 26266, 1035, 1035, 1999, 4183, 1035, 1035, 1006, 1007, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokens = tokenizer.tokenize(text) 
rich.print ( tokens)

[
    'what',
    'does',
    "'",
    'super',
    "'",
    'do',
    'in',
    'python',
    '?',
    '-',
    'difference',
    'between',
    'super',
    '(',
    ')',
    '.',
    '_',
    '_',
    'in',
    '##it',
    '_',
    '_',
    '(',
    ')',
    'and',
    'explicit',
    'super',
    '##class',
    '_',
    '_',
    'in',
    '##it',
    '_',
    '_',
    '(',
    ')'
]

In [10]:
input_ids= tokenizer.convert_tokens_to_ids(tokens)
input_ids

[2054,
 2515,
 1005,
 3565,
 1005,
 2079,
 1999,
 18750,
 1029,
 1011,
 4489,
 2090,
 3565,
 1006,
 1007,
 1012,
 1035,
 1035,
 1999,
 4183,
 1035,
 1035,
 1006,
 1007,
 1998,
 13216,
 3565,
 26266,
 1035,
 1035,
 1999,
 4183,
 1035,
 1035,
 1006,
 1007]

In [11]:
decoded_string = tokenizer.decode(input_ids)
decoded_string

"what does'super'do in python? - difference between super ( ). _ _ init _ _ ( ) and explicit superclass _ _ init _ _ ( )"

## Dset

In [12]:
dset = datasets.load_from_disk("../data/processed")

In [13]:
dset

Dataset({
    features: ['photo_id', 'photo_url', 'photo_image_url', 'photo_submitted_at', 'photo_featured', 'photo_width', 'photo_height', 'photo_aspect_ratio', 'photo_description', 'photographer_username', 'photographer_first_name', 'photographer_last_name', 'exif_camera_make', 'exif_camera_model', 'exif_iso', 'exif_aperture_value', 'exif_focal_length', 'exif_exposure_time', 'photo_location_name', 'photo_location_latitude', 'photo_location_longitude', 'photo_location_country', 'photo_location_city', 'stats_views', 'stats_downloads', 'ai_description', 'ai_primary_landmark_name', 'ai_primary_landmark_latitude', 'ai_primary_landmark_longitude', 'ai_primary_landmark_confidence', 'blur_hash', 'description_final', 'image'],
    num_rows: 24995
})

In [14]:
rich.print ( dset[0] )

{
    'photo_id': 'XMyPniM9LF0',
    'photo_url': 'https://unsplash.com/photos/XMyPniM9LF0',
    'photo_image_url': 'https://images.unsplash.com/uploads/14119492946973137ce46/f1f2ebf3',
    'photo_submitted_at': '2014-09-29 00:08:38.594364',
    'photo_featured': 't',
    'photo_width': 4272,
    'photo_height': 2848,
    'photo_aspect_ratio': 1.5,
    'photo_description': 'Woman exploring a forest',
    'photographer_username': 'michellespencer77',
    'photographer_first_name': 'Michelle',
    'photographer_last_name': 'Spencer',
    'exif_camera_make': 'Canon',
    'exif_camera_model': 'Canon EOS REBEL T3',
    'exif_iso': 400.0,
    'exif_aperture_value': '1.8',
    'exif_focal_length': '50.0',
    'exif_exposure_time': '1/100',
    'photo_location_name': None,
    'photo_location_latitude': None,
    'photo_location_longitude': None,
    'photo_location_country': None,
    'photo_location_city': None,
    'stats_views': 2375421,
    'stats_downloads': 6967,
    'ai_description': 'woman walking in the middle of forest',
    'ai_primary_landmark_name': None,
    'ai_primary_landmark_latitude': None,
    'ai_primary_landmark_longitude': None,
    'ai_primary_landmark_confidence': None,
    'blur_hash': 'L56bVcRRIWMh.gVunlS4SMbsRRxr',
    'description_final': 'Woman exploring a forest',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427 at 0x7FA46B477370>
}

In [15]:
corpus = model.encode(dset['description_final'])

In [19]:
def find_results(query:str , k =5):
    
    
    query_features = model.encode(query)
    doc_scores = query_features @ corpus.T

    top_items = doc_scores.argsort()[-k:][::-1]



    degug_info = {
         "query_original":  query 
         , "query_processed" : query_features
        , "doc_scores":  doc_scores 
        , "top_items":  top_items 
    }
    rich.print (degug_info )
    

    display(HTML(f"<h1>Query: {query} </h1>"))
    
    # Iterate over the top k results
    for idx, photo_data in enumerate( dset.select(top_items)):

        doc_idx = top_items[idx]
        # Display the photo
        display(Image(url=photo_data["photo_image_url"] + "?w=200"))

        # Display the attribution text
        display(HTML(f"""
                     Photo title: {photo_data["description_final"]}   <br/>
                     Photo by <a href="https://unsplash.com/@{photo_data["photographer_username"]}?utm_source=NaturalLanguageImageSearch&utm_medium=referral">{photo_data["photographer_first_name"]} {photo_data["photographer_last_name"]}</a> on <a href="https://unsplash.com/?utm_source=SearchWorkshop&utm_medium=referral">Unsplash</a> <br/>
                     Distance: {doc_scores[doc_idx]}
                     """
                                        ))
        print()

In [20]:
find_results( "Two dogs playing in the snow")

{
    'query_original': 'Two dogs playing in the snow',
    'query_processed': array([-3.45207192e-02, -5.31195141e-02,  7.89919794e-02,  3.99201699e-02,
       -6.76747635e-02,  2.95135509e-02, -4.12319116e-02,  3.39880660e-02,
        4.66178693e-02,  7.46874288e-02, -2.96282247e-02,  1.31358057e-02,
        2.92284675e-02,  5.03163859e-02,  4.19280455e-02,  1.21763516e-02,
       -7.26377368e-02, -5.97697273e-02,  1.40849371e-02, -3.02251168e-02,
       -2.28216611e-02, -7.01758042e-02, -3.69652919e-02, -2.73013068e-03,
       -1.33206332e-02,  8.86703376e-03, -5.50668538e-02,  3.70550007e-02,
       -2.70354338e-02,  2.35992819e-02, -3.81438583e-02, -2.63989381e-02,
       -8.23715888e-03,  3.31010483e-03, -3.23406383e-02, -7.06446264e-03,
        6.74610883e-02,  3.86140160e-02,  4.72208150e-02, -4.19445857e-02,
        1.86015014e-02,  7.76332244e-03,  9.32705414e-04, -8.11238438e-02,
       -4.81196120e-02, -1.15978289e-02, -7.39517063e-02,  3.85072120e-02,
        3.34395766e-02,  3.75415981e-02, -1.82045773e-02,  6.45072246e-03,
       -5.12047634e-02,  5.35726845e-02, -5.31552732e-02,  2.76665832e-03,
       -3.48396711e-02, -1.28425695e-02, -1.80092584e-02, -2.29026489e-02,
        4.58910614e-02,  2.29173861e-02,  2.25099828e-02,  5.52725010e-02,
        2.96301600e-02, -9.61857196e-03, -1.15360046e-04, -2.04836559e-02,
       -5.97529523e-02, -3.31123061e-02, -1.68398879e-02,  5.59742525e-02,
        4.34544086e-02, -1.03060447e-01, -2.36169714e-02, -5.89062087e-03,
       -6.88845217e-02, -2.29419488e-02,  2.27818917e-03, -1.66133493e-02,
        7.57169165e-03, -1.38054201e-02, -5.44165121e-03,  1.57766696e-02,
        6.21815659e-02, -7.71509530e-03,  3.56242694e-02, -8.37733131e-03,
       -4.72189300e-03, -7.33623058e-02, -7.27195218e-02, -9.11195949e-02,
       -6.61582081e-03,  4.94386666e-02, -3.40759493e-02, -3.86048667e-02,
        2.50481013e-02, -1.37743615e-02,  2.61266176e-02,  1.92103125e-02,
       -5.63198514e-03, -7.35356733e-02,  2.19636597e-02, -2.64104847e-02,
        1.31724998e-01,  2.08567344e-02, -8.08987319e-02,  4.44578938e-03,
        7.32471282e-03,  4.33451012e-02,  2.01865677e-02, -2.47319136e-02,
       -1.38191776e-02,  7.01132864e-02,  8.05389043e-03,  1.43697374e-02,
       -4.59568799e-02, -5.13289636e-03, -3.29643637e-02, -2.43084226e-02,
        1.00730015e-02,  8.00301731e-02, -2.46462002e-02, -2.32016146e-02,
        8.80044103e-02, -7.34434798e-02,  9.53080039e-03, -5.51798207e-33,
        6.67391717e-02, -8.99631009e-02, -6.39788210e-02, -4.10474762e-02,
        2.60285083e-02, -1.71097405e-02,  7.28479400e-03,  7.89155625e-03,
        3.16862017e-02,  6.22459641e-03, -3.96656319e-02,  8.47623721e-02,
       -4.32999048e-04, -1.22053288e-01,  9.50080827e-02, -2.76546031e-02,
        2.67648064e-02, -3.20277251e-02,  8.78301561e-02,  4.40087207e-02,
       -1.33216437e-02,  7.61411563e-02,  6.68609738e-02,  6.21849410e-02,
        4.16374058e-02,  5.30747836e-03, -2.84969024e-02, -7.92078599e-02,
       -7.50016272e-02, -4.21432517e-02,  7.64279217e-02, -9.42026749e-02,
       -2.36884039e-02,  7.31200054e-02,  3.73845622e-02,  3.13345715e-03,
       -7.00921938e-03, -5.84262572e-02,  4.34534699e-02, -1.55279301e-02,
        8.30216706e-02, -8.67197588e-02,  1.93145666e-02, -6.86870217e-02,
       -1.71579327e-02, -4.59659733e-02, -6.00139312e-02, -1.61339119e-02,
       -4.52781916e-02, -4.90354449e-02,  9.49769244e-02,  6.90665692e-02,
        1.16112819e-02,  1.89545024e-02,  5.50991893e-02,  2.47586947e-02,
        5.46792671e-02, -2.11137608e-02, -1.58346724e-02,  4.82045002e-02,
        2.42203046e-02, -2.96652131e-02,  3.70465741e-02, -1.28934279e-01,
        3.76302712e-02, -6.83029741e-02,  6.02344275e-02,  9.30509567e-02,
       -5.75887412e-02, -9.55608953e-03,  8.76331478e-02,  7.40666990e-04,
        1.05235621e-01, -7.96228871e-02,  2.33249050e-02, -1.55287990e-02,
        3.85891423e-02, -1.05712786e-01,  8.39892998e-02, -1.91817747e-03,
        1.38400858e-0

In [21]:
find_results( "boy and girl on a beach")

{
    'query_original': 'boy and girl on a beach',
    'query_processed': array([-4.07104753e-02,  4.79862355e-02,  2.29214653e-02, -4.88387793e-02,
        2.43954528e-02, -2.38477904e-02,  3.11611686e-02, -1.36817796e-02,
        4.65978794e-02, -9.23146363e-05,  1.02531508e-01, -9.68877226e-02,
        5.65099232e-02,  3.85042578e-02,  7.72961676e-02,  7.91627914e-02,
        1.92165114e-02, -1.42192829e-03, -4.85068150e-02, -2.43043993e-02,
       -9.99627635e-02,  4.58659651e-03, -7.73032233e-02,  1.18157025e-02,
       -6.53491542e-02,  4.79752533e-02,  7.68425316e-02, -1.17027899e-02,
       -7.49083757e-02,  4.83290516e-02,  3.65568027e-02, -3.88130769e-02,
       -5.47474343e-03, -2.54936460e-02,  7.50967825e-04,  4.39735502e-02,
       -5.32944538e-02, -6.88708499e-02, -2.03870647e-02, -5.97221740e-02,
       -9.35861841e-02, -2.68354267e-02,  6.48142828e-04,  9.61101893e-03,
       -2.36955136e-02, -6.07894473e-02,  9.62193087e-02,  4.55094837e-02,
        8.98922458e-02,  3.65537964e-02, -3.67548363e-03, -8.01947713e-02,
       -7.77877420e-02,  5.63851148e-02, -6.62852749e-02, -5.95939055e-04,
       -6.77137375e-02, -6.88764304e-02,  5.58785610e-02,  3.94712426e-02,
       -1.03117786e-02,  6.33546710e-02,  4.76702722e-03,  1.48585262e-02,
        6.33022860e-02,  2.51070447e-02,  2.42034905e-02, -4.33537513e-02,
        1.12408539e-02,  2.39848606e-02, -3.95492949e-02,  3.48774903e-02,
        4.42005880e-02,  3.65096368e-02, -6.26903400e-02,  9.37430281e-03,
       -2.34903116e-02,  6.74278364e-02,  2.77797114e-02, -9.90912020e-02,
       -9.37623158e-02, -1.04982391e-01,  1.68256946e-02,  1.95917627e-03,
        7.06340233e-03, -2.34424137e-02, -1.62639413e-02,  7.84643833e-03,
       -3.76534313e-02,  5.02219982e-03, -9.07532424e-02, -2.25280821e-02,
        3.72708589e-02, -1.27724409e-02, -1.90754998e-02, -4.87265773e-02,
       -6.75674528e-02, -3.08920965e-02, -3.49576101e-02,  5.04678078e-02,
        4.59770393e-03,  1.22856341e-01,  2.11283397e-02,  3.66596505e-02,
       -6.05403446e-03, -9.67195537e-03,  2.55508162e-02, -4.37193364e-02,
        5.63997850e-02,  2.45086905e-02,  1.04546535e-03,  1.45639889e-02,
        2.35099848e-02,  4.33364362e-02, -7.37936199e-02, -3.24158045e-03,
        1.17685169e-01, -5.81584238e-02, -7.67244995e-02, -4.78574112e-02,
        3.65472287e-02, -7.93343633e-02,  1.47296023e-02, -1.09353801e-02,
       -7.76054943e-03, -8.79522637e-02,  2.16943286e-02, -3.39482769e-33,
        3.77268307e-02, -7.57371821e-03,  5.71781136e-02,  4.15935926e-02,
        1.57682318e-02,  6.73608063e-03,  1.47388363e-02, -4.78137694e-02,
       -3.16880457e-02, -4.88858297e-02, -1.14096768e-01, -9.58325639e-02,
       -2.90886927e-02, -4.33061942e-02,  1.07437268e-01, -6.02333844e-02,
        2.01424584e-02, -5.47444820e-02, -8.73425603e-02,  9.99743268e-02,
        3.84692885e-02,  5.23424894e-02, -1.05295582e-02, -3.25250588e-02,
       -3.72182956e-04,  2.35905647e-02,  1.16148070e-02,  5.49267083e-02,
        1.59943067e-02,  1.17658200e-02,  5.73256873e-02, -4.38547134e-02,
        5.54436026e-03,  4.54594456e-02, -4.31809295e-03, -2.07610764e-02,
        7.72089064e-02,  6.89690467e-03,  4.05697748e-02,  1.39073851e-02,
        3.15449685e-02, -4.53322157e-02, -3.96225788e-03,  2.50627846e-02,
       -3.23374383e-02, -5.69311976e-02,  2.09061103e-03,  1.80730801e-02,
       -1.88388210e-02, -1.14544496e-01, -3.08165103e-02, -2.04545911e-02,
       -6.48095384e-02,  5.24386279e-02,  2.18987887e-04,  5.02021648e-02,
        4.24139295e-03, -3.70574147e-02, -1.62888750e-01,  6.86493888e-03,
        6.48717061e-02, -2.45465878e-02,  2.60955077e-02, -3.98829430e-02,
       -4.38777395e-02,  8.39287043e-02,  1.27761319e-01,  5.83266392e-02,
       -2.32847757e-03, -1.12390116e-01, -4.09222506e-02,  1.63831711e-02,
        1.19267292e-01, -6.34890236e-03,  1.25049418e-02,  3.41344886e-02,
        5.40477373e-02, -1.22568561e-02, -1.32459989e-02,  8.55198726e-02,
       -2.82161850e-02,  3

In [22]:
find_results( "image of a man in a desert")

{
    'query_original': 'image of a man in a desert',
    'query_processed': array([ 6.32947534e-02,  1.24192499e-01, -2.32294332e-02,  7.78998807e-03,
        2.82716788e-02, -2.40863636e-02,  1.31348416e-01, -6.79284930e-02,
        2.57225148e-02,  1.04728043e-02,  5.06874435e-02, -8.11643004e-02,
        9.45465490e-02,  5.87001480e-02,  5.51839098e-02, -3.88276018e-02,
       -3.57096419e-02,  3.80245745e-02,  3.86204235e-02, -2.83764917e-02,
       -2.16995738e-02,  2.87901442e-02,  1.18983900e-02, -5.04343174e-02,
       -7.90204480e-02, -2.33318489e-02, -9.52090323e-03,  5.76721951e-02,
       -2.41583139e-02, -3.99373621e-02,  3.81454490e-02,  2.18772311e-02,
        2.12652571e-02, -3.48423868e-02,  1.09066674e-02,  1.06592728e-02,
       -5.02256118e-02,  3.19877379e-02, -5.95463961e-02,  4.30590175e-02,
        2.06208210e-02, -4.87505533e-02,  5.11444099e-02,  5.83226718e-02,
        2.46753171e-02,  6.96470886e-02,  3.27836117e-03, -1.08690711e-03,
        1.23149894e-01,  8.28368030e-03, -8.92766044e-02,  8.87478702e-03,
       -4.88979481e-02, -5.94949797e-02,  2.77780415e-03, -5.36132045e-02,
        4.49112989e-02, -2.03460325e-02, -3.06635741e-02, -3.85979414e-02,
        1.73355006e-02,  3.44189741e-02, -1.35636434e-03,  1.07655842e-05,
        1.40614912e-01, -3.98737937e-02,  1.58259389e-03, -1.16327114e-01,
        5.28831482e-02, -6.69457465e-02, -1.00005390e-02,  9.94998217e-03,
       -2.71171369e-02, -8.05303901e-02, -1.15467973e-01, -1.03774965e-01,
       -3.83616276e-02,  1.20942928e-02, -3.43221910e-02, -5.17199701e-03,
        9.89541784e-02, -6.89451629e-03,  3.88789997e-02, -4.81113978e-02,
       -2.26192009e-02,  1.60770249e-02, -4.23194980e-03,  1.82598026e-03,
        2.06454583e-02,  5.77857625e-03, -4.65922542e-02, -4.71361987e-02,
       -4.41993885e-02,  5.06741635e-04, -4.62268554e-02, -1.73499938e-02,
        2.00978643e-03, -3.05878092e-02, -6.82516992e-02,  6.81843162e-02,
        8.01285952e-02, -1.06879808e-01,  1.22480812e-02, -2.52861977e-02,
        1.05119042e-01, -1.24540552e-02,  6.40390953e-03,  1.77443493e-02,
       -3.00352704e-02, -9.96275991e-03, -3.68016362e-02, -6.61875010e-02,
       -4.56597582e-02,  2.86053885e-02,  6.47853464e-02, -2.54829507e-02,
       -5.31485118e-02, -6.33666813e-02, -6.67340606e-02,  1.94186717e-02,
       -1.15128644e-02,  3.60787250e-02,  2.06134655e-02,  9.24573466e-02,
        3.93706039e-02, -1.23830661e-02,  5.65089807e-02, -5.97997853e-33,
        3.33683603e-02,  3.97037789e-02,  5.59407584e-02,  4.35199626e-02,
        5.27929440e-02, -2.16232762e-02,  4.09237714e-03, -7.75826648e-02,
       -2.97988057e-02, -7.98005462e-02,  5.02836369e-02, -1.58946309e-02,
       -2.43129078e-02,  1.61693841e-01, -3.71846594e-02, -1.35098230e-02,
       -4.57702111e-03, -5.40835559e-02,  1.45904589e-02, -3.94143052e-02,
       -7.38223642e-02,  3.93503830e-02, -2.17185845e-03, -3.48750055e-02,
        2.12167185e-02,  3.35267149e-02,  7.90317543e-03, -3.54881920e-02,
        2.25763228e-02,  2.97231018e-03, -1.76198047e-03,  4.03513089e-02,
        7.02505093e-03, -4.67241965e-02,  2.30240431e-02, -5.00222109e-02,
       -5.43840006e-02,  5.41807376e-02, -7.89700523e-02,  1.95282158e-02,
        1.05172740e-02,  2.44100019e-02,  6.15005828e-02,  2.93591619e-02,
        2.17619725e-02, -2.54875124e-02,  5.29429801e-02,  9.41745341e-02,
       -1.48999076e-02,  5.13313012e-03, -2.50351634e-02,  3.08064204e-02,
        9.95937642e-03, -7.74944648e-02, -6.28233775e-02, -1.42882504e-02,
        5.68795204e-02, -1.67584028e-02, -9.76467356e-02,  1.92010738e-02,
        5.68931624e-02, -6.54330198e-03, -2.20090393e-02,  3.55685800e-02,
        1.49378534e-02, -5.76085374e-02, -3.87795009e-02,  1.49252620e-02,
       -4.35932837e-02,  1.27165858e-02, -2.35435646e-02,  3.54422703e-02,
        7.37417191e-02, -1.69062410e-02, -5.55281751e-02,  2.36016903e-02,
        3.10728885e-02,  9.29200370e-03, -1.39807940e-01,  6.31070510e-02,
       -8.65007564e-02,

## timing

In [23]:
%%timeit
search_query = "Two dogs playing in the snow"
k=5
text_features = model.encode(search_query)
doc_scores = text_features @ corpus.T

top_items = doc_scores.argsort()[-k:][::-1]


44.6 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
